In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.bmi.replace(to_replace=np.nan, value=df.bmi.mean(), inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:

df['bmi_cat'] = pd.cut(df['bmi'], bins = [0, 19, 25,30,10000], labels = ['Underweight', 'Ideal', 'Overweight', 'Obesity'])
df['age_cat'] = pd.cut(df['age'], bins = [0,13,18, 45,60,200], labels = ['Children', 'Teens', 'Adults','Mid Adults','Elderly'])
df['glucose_cat'] = pd.cut(df['avg_glucose_level'], bins = [0,90,160,230,500], labels = ['Low', 'Normal', 'High', 'Very High'])

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['bmi','id'])

In [ ]:
df

In [ ]:
plt.figure(figsize=(10,8))
sns.kdeplot(data = df[df['stroke'] == 0], x = 'age', shade = True,  alpha = 0.8, color = 'blue',Label='NOSTROKE')
sns.kdeplot(data = df[df['stroke'] == 1], x = 'age', shade = True,  alpha = 0.8, color = 'red',Label='STROKE')
plt.legend()

In [ ]:
plt.figure(figsize=(10,8))
sns.kdeplot(data = df[df['stroke'] == 0], x = 'avg_glucose_level', shade = True,  alpha = 1, color = '#512b58',Label='NOSTROKE' )
sns.kdeplot(data = df[df['stroke'] == 1], x = 'avg_glucose_level', shade = True,  alpha = 0.8, color = '#fe346e',Label='STROKE')
plt.legend()


In [ ]:
plt.figure(figsize=(10,8))
sns.kdeplot(data = df[df['stroke'] == 0], x = 'bmi', shade = True,  alpha = 1, color = '#512b58',Label='NOSTROKE' )
sns.kdeplot(data = df[df['stroke'] == 1], x = 'bmi', shade = True,  alpha = 0.8, color = '#fe346e',Label='STROKE')
plt.legend()


In [ ]:
healthy_gen = df[df['stroke'] == 0].gender.value_counts()
stroke_gen = df[df['stroke'] == 1].gender.value_counts()
plt.bar( healthy_gen.index , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')


In [ ]:
healthy_gen = df[df['stroke'] == 0].work_type.value_counts()
stroke_gen = df[df['stroke'] == 1].work_type.value_counts()
plt.bar( healthy_gen.index , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')


In [ ]:
healthy_gen = df[df['stroke'] == 0].Residence_type.value_counts()
stroke_gen = df[df['stroke'] == 1].Residence_type.value_counts()
plt.bar( healthy_gen.index , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')


In [ ]:
healthy_gen = df[df['stroke'] == 0].smoking_status.value_counts()

stroke_gen = df[df['stroke'] == 1].smoking_status.value_counts()
plt.bar( healthy_gen.index , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')


In [ ]:
healthy_gen = df[df['stroke'] == 0].hypertension.value_counts()

stroke_gen = df[df['stroke'] == 1].hypertension.value_counts()
plt.bar( ['YES','NO'] , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')
plt.xlabel('HYPERTENSION')

In [ ]:
healthy_gen = df[df['stroke'] == 0].heart_disease.value_counts()

stroke_gen = df[df['stroke'] == 1].heart_disease.value_counts()
plt.bar( ['YES','NO'] , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')
plt.xlabel('HEARTDISEASE')

In [ ]:
healthy_gen = df[df['stroke'] == 0].ever_married.value_counts()

stroke_gen = df[df['stroke'] == 1].ever_married.value_counts()
plt.bar( ['YES','NO'] , height = healthy_gen.values, width = 0.5, color = 'blue')
plt.bar( np.arange(len(stroke_gen.index)), height = stroke_gen.values, width = 0.5, color= 'red')
plt.xlabel('MARRITIALRISK')

In [ ]:
df.head()
a=df.drop(columns=['id','bmi_cat','age_cat','glucose_cat'])
a.head()

In [ ]:
x = a.iloc[ :,0:-1].values
y = a.iloc[:, -1].values


In [ ]:
x

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [0,5,9])], remainder= 'passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 15] = le.fit_transform(x[:, 15])
x[:, 16] = le.fit_transform(x[:, 16])

In [ ]:
x.shape


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 0)

In [ ]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score


In [ ]:
models = []
models.append(['Logistic Regreesion', LogisticRegression(random_state=0)])

models.append(['KNeighbors', KNeighborsClassifier()])


models.append(['Random Forest', RandomForestClassifier(random_state=0)])

lst_1= []

for m in range(len(models)):
    lst_2= []
    model = models[m][1]
    model.fit(x_train_res, y_train_res)
    y_pred = model.predict(x_test)
    cm = confusion_matrix(y_test, y_pred)  #Confusion Matrix
    accuracies = cross_val_score(estimator = model, X = x_train_res, y = y_train_res, cv = 10)   #K-Fold Validation
    roc = roc_auc_score(y_test, y_pred)  #ROC AUC Score
    precision = precision_score(y_test, y_pred)  #Precision Score
    recall = recall_score(y_test, y_pred)  #Recall Score
    f1 = f1_score(y_test, y_pred)  #F1 Score
    print(models[m][0],':')
    print(cm)
    print('Accuracy Score: ',accuracy_score(y_test, y_pred))
    print('')
    print("K-Fold Validation Mean Accuracy: {:.2f} %".format(accuracies.mean()*100))
    print('')
    print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
    print('')
    print('ROC AUC Score: {:.2f}'.format(roc))
    print('')
    print('Precision: {:.2f}'.format(precision))
    print('')
    print('Recall: {:.2f}'.format(recall))
    print('')
    print('F1: {:.2f}'.format(f1))
    print('-----------------------------------')
    print('')
    lst_2.append(models[m][0])
    lst_2.append((accuracy_score(y_test, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_2.append(accuracies.std()*100)
    lst_2.append(roc)
    lst_2.append(precision)
    lst_2.append(recall)
    lst_2.append(f1)
    lst_1.append(lst_2)

In [ ]:

from sklearn.model_selection import GridSearchCV
grid_models = [(LogisticRegression(),[{'C':[0.25,0.5,0.75,1],'random_state':[0]}]), 
               (KNeighborsClassifier(),[{'n_neighbors':[5,7,8,10], 'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}]), 
                (RandomForestClassifier(),[{'n_estimators':[100,150,200],'criterion':['gini','entropy'],'random_state':[0]}])]

In [ ]:
for i,j in grid_models:
    grid = GridSearchCV(estimator=i,param_grid = j, scoring = 'accuracy',cv = 10)
    grid.fit(x_train_res, y_train_res)
    best_accuracy = grid.best_score_
    best_param = grid.best_params_
    print('{}:\nBest Accuracy : {:.2f}%'.format(i,best_accuracy*100))
    print('Best Parameters : ',best_param)
    print('')
    print('----------------')
    print('')